Dependencies

docx: pip install python-docx

PyPDF2: pip install pypdf2

NLTK  : pip install -U nltk


In [52]:
import glob2
import os
import docx
from docx import Document
import PyPDF2
import csv
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
#Check if have download all needed additional data for nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to C:\Users\maxpr/nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

### Openning files

In [3]:
keppels_files = glob2.glob('files/Keppel/*')
prudential_files = glob2.glob('files/Prudential/*')
print(f'we have {len(keppels_files)} keppels documents and {len(prudential_files)} Prudential documents')

we have 193 keppels documents and 194 Prudential documents


Let's check the percentage of different document type

In [4]:
keppels_files_pdf = glob2.glob('files/Keppel/*.pdf')
keppels_files_xls = glob2.glob('files/Keppel/*.xlsx') + glob2.glob('files/Keppel/*.xls')
keppels_files_doc = glob2.glob('files/Keppel/*.docx') + glob2.glob('files/Keppel/*.doc')
len(keppels_files_pdf) + len(keppels_files_xls) + len(keppels_files_doc)

193

In [5]:
prudential_files_pdf = glob2.glob('files/Prudential/*.pdf')
prudential_files_xls = glob2.glob('files/Prudential/*.xlsx') + glob2.glob('files/Prudential/*.xls')
prudential_files_doc = glob2.glob('files/Prudential/*.docx') + glob2.glob('files/Prudential/*.doc')
len(prudential_files_pdf) + len(prudential_files_xls) + len(prudential_files_doc)

194

From that we can conclude that the documents are of three different types (pdf, word and excel)

In [6]:
doc = Document(keppels_files_doc[0])
def getText(filename):
    if(filename.endswith("docx")):
        doc = docx.Document(filename)
    else:
        doc = doc.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

In [7]:
print(f'File {keppels_files_doc[3]} content is :')
print((getText(keppels_files_doc[3])))

File files/Keppel\CYP%20Meeting%20May%202017docx.docx content is :
CHILDREN AND YOUNG PEOPLE NETWORK
18.05.2017 at London Funders

Participants 
         Deborah	Meyer	Big Lottery Fund 
	Toyin	Ogundana	CAF Venturesome
	Daisy	Ryan	Henry Smith Charity
	Erik	Mesel	John Lyon's Charity
	Sharon	Long	Partnership For Young London
	Mihai	Calin Bica	Roma Support Group
	Gaba	Smolinska-	Roma Support Group
	Poffley
	Victoria	Cousins	Safer London Foundation 
	Cara	Whelan	Save the Children
	Emma	Hale	Sir John Cass's Foundation
	Rachel	Earnshaw	The Big Give
	Ciaran	Rafferty	The City Bridge Trust
	Jenny	Oppenheimer	The Pilgrim Trust
	Patrick	Kinsella	Traveller Movement
	Yvonne	MacNamara	Traveller Movement
	Flora	Ward	Youth Music

In attendance
         Geraldine	Tovey	London Funders
	David	Warner	London Funders

Welcome and apologies for absence

Ciaran welcomed everyone to the group and introductions were made. He welcomed the group to its first network meeting in 2017 following the Vision for Young L

In [8]:
#keep : no not don't aren't aren
ourStopWord = ['i', 'me', 'my', 'myself', 'we', 'our','I',
               'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
               'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
               'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs',
               'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those',
               'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do',
               'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
               'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
               'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on',
               'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where',
               'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such',
                'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will',
               'just','should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y',
                'ma' ]


In [9]:
def removeStop(text):
    stop_words = ourStopWord
 
    word_tokens = word_tokenize(text)
 
    #filtered_sentence = [w for w in word_tokens if not w in stop_words]
 
    filtered_sentence = []
 
    for w in word_tokens:
        #w = w.lower()
        if w not in stop_words:
            if (w=="n\'t"):
                w="not"
            filtered_sentence.append(w)
    return filtered_sentence

In [10]:
text = removeStop(getText(keppels_files_doc[3]))

In [11]:
def textPOS(text):
    #text = [c.lower() for c in text]
    return nltk.pos_tag(text)

In [12]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [13]:
textCleaned = removeStop(getText(keppels_files_doc[3]))

In [14]:
textPOS = textPOS(textCleaned)

In [15]:
textPOS[100:1000]

[('Funders', 'NNP'),
 ('Welcome', 'NNP'),
 ('apologies', 'NNS'),
 ('absence', 'NN'),
 ('Ciaran', 'NNP'),
 ('welcomed', 'VBD'),
 ('everyone', 'NN'),
 ('group', 'NN'),
 ('introductions', 'NNS'),
 ('made', 'VBD'),
 ('.', '.'),
 ('He', 'PRP'),
 ('welcomed', 'VBD'),
 ('group', 'NN'),
 ('first', 'RB'),
 ('network', 'NN'),
 ('meeting', 'NN'),
 ('2017', 'CD'),
 ('following', 'VBG'),
 ('Vision', 'NNP'),
 ('Young', 'NNP'),
 ('Londoners', 'NNPS'),
 ('event', 'NN'),
 ('earlier', 'RB'),
 ('year', 'NN'),
 ('.', '.'),
 ('Report', 'NNP'),
 ('last', 'JJ'),
 ('meeting', 'NN'),
 (',', ','),
 ('30th', 'CD'),
 ('November', 'NNP'),
 ('2016', 'CD'),
 ('Minutes', 'NNP'),
 ('last', 'JJ'),
 ('Children', 'NNP'),
 ('Young', 'NNP'),
 ('People', 'NNP'),
 ('’', 'NNP'),
 ('meeting', 'VBG'),
 ('agreed', 'VBD'),
 ('.', '.'),
 ('Yvonne', 'NNP'),
 ('MacNamara', 'NNP'),
 ('Patrick', 'NNP'),
 ('Kinsella-', 'NNP'),
 ('Traveller', 'NNP'),
 ('Movement', 'NNP'),
 ('The', 'DT'),
 ('Traveller', 'NNP'),
 ('Movement', 'NNP'),
 ('s

In [16]:
def lemmize(text):
    wordnet_lemmatizer = WordNetLemmatizer()
    finalStemmed = []
    for w in text : 
        allInfo = w
        word = allInfo[0]
        tag = allInfo[1]
        stemmedWord = ""
        try:
            stemmedWord = wordnet_lemmatizer.lemmatize(word,get_wordnet_pos(tag))
            finalStemmed.append((stemmedWord,w[1][0]))
        except Exception:
            finalStemmed.append((w[0],w[1][0]))     
    return finalStemmed

In [17]:
print(lemmize(textPOS[:100]))

[('CHILDREN', 'N'), ('AND', 'C'), ('YOUNG', 'N'), ('PEOPLE', 'N'), ('NETWORK', 'N'), ('18.05.2017', 'C'), ('London', 'N'), ('Funders', 'N'), ('Participants', 'N'), ('Deborah', 'N'), ('Meyer', 'N'), ('Big', 'N'), ('Lottery', 'N'), ('Fund', 'N'), ('Toyin', 'N'), ('Ogundana', 'N'), ('CAF', 'N'), ('Venturesome', 'N'), ('Daisy', 'N'), ('Ryan', 'N'), ('Henry', 'N'), ('Smith', 'N'), ('Charity', 'N'), ('Erik', 'N'), ('Mesel', 'N'), ('John', 'N'), ('Lyon', 'N'), ("'s", 'P'), ('Charity', 'N'), ('Sharon', 'N'), ('Long', 'N'), ('Partnership', 'N'), ('For', 'I'), ('Young', 'N'), ('London', 'N'), ('Mihai', 'N'), ('Calin', 'N'), ('Bica', 'N'), ('Roma', 'N'), ('Support', 'N'), ('Group', 'N'), ('Gaba', 'N'), ('Smolinska-', 'N'), ('Roma', 'N'), ('Support', 'N'), ('Group', 'N'), ('Poffley', 'N'), ('Victoria', 'N'), ('Cousins', 'N'), ('Safer', 'N'), ('London', 'N'), ('Foundation', 'N'), ('Cara', 'N'), ('Whelan', 'N'), ('Save', 'N'), ('Children', 'N'), ('Emma', 'N'), ('Hale', 'N'), ('Sir', 'N'), ('John', '

In [18]:
print(keppels_files_pdf[84])
pdf = PyPDF2.PdfFileReader(keppels_files_pdf[4],'rb')
pdf.getNumPages()
print(pdf.getPage(6).extractText())

files/Keppel\KLL_AR2002a.pdf
 Sustaining Growth 7Keppel Land, whose selective capital reduction received the approval of participating shareholders at its Extraordinary General Meeting on 
 13 April 2016, contributed to 47% of the Group™s earnings in the ˜rst quarter of this year. Pictured is the Keppel Land development, 
 Highline Residences in Tiong Bahru
and an approximate 27% 
premium over its latest 

valuation.  Tapping on demand in 
recovering property markets 

across Asia, Keppel Land has 

about 20,000 homes in its 
portfolio ready for launch 
over the next two years, 

mostly in China.In parallel, Keppel Land is 
actively strengthening its 

portfolio of commercial 

properties which has 
increased to over one 
 million sm of gross ˚oor area. 
INFRASTRUCTURE Energy Infrastructure & 
Services
Our team is pursuing growth 

opportunities in key areas 
such as Gas-to-Power and 
WTE both in Singapore 

and overseas. They are 

focused on building Keppel 
Infrastructure into a sta

In [21]:
pdf = PyPDF2.PdfFileReader("files/Keppel\\2017-DeepwaterPoster-D6-Ads.pdf",'rb')
print(pdf.getPage(0).extractText())

Information Accuracy: 
Every attempt has been made to identify, locate, and 
establish the most current and correct information for all the various types 
of deepwater production solutions either sanctioned, installed, operating or 
decommissioned worldwide. No facility was intentionally excluded from this 

poster.  In some cases the most current information was not included because 
information was not supplied in time or the project was not sanctioned at 
press time.  We make no guarantee that this list is all inclusive.  We have 
summarized the capability and operating experience by acting as a neutral 
party and integrator of information. We have collected the information from 

company brochures, interviews, press releases, industry publications, contractor 
supplied information, and websites. Neither Wood 
Group nor Offshore Magazine 
guarantees or assumes 

any responsibility or liability for any reference on the 
information presented.  If any information is found to 

be inco

In [19]:
def countPDF(pdf,comp):
    count = 0
    result = []
    count = 0
    try:
        doc = PyPDF2.PdfFileReader(pdf,'r')
        index = doc.getNumPages()
        #print(index)
        if(index<100):
            for i in range(0,index):
                txt = doc.getPage(i).extractText()
                allS = txt.split(' ')
                for s in allS:
                    if(comp in s.lower()):
                        count+=1
    except:
        print("COULD NOT OPEN THE PDF ", pdf)
    return count

In [20]:
def countDoc(text,comp):
    count = 0
    try :
        txt = docx2txt.process(text)
        print(txt[:50])
        allS = txt.split(' ')
        for s in allS:
            if(comp in s.lower()):
                    count+=1
    except:
        print("COULD NOT OPEN DOC(X)", text )
    return count

In [21]:
def mapperOpener(path):
    mapping = []
    with open(path, 'r') as csvfile:
        spamreader = csv.reader(csvfile,delimiter='|')
        for row in spamreader:
            mapping.append((row[0],row[1],row[2]))
    return mapping

In [22]:
filesMap = mapperOpener("submission_mapper.csv")

In [68]:
for c in filesMap:
    if(c[1].endswith('doc')):
        print(c[1] + " " + c[2])

sin63964.doc Keppel
bmap_text_8.doc Keppel
Dr%20Rob%20Kelly%20-%20Keppel%20Fels%20Loadout%20Speech.doc Keppel
STAT%20379s08%20study%20guide.doc Keppel
TPG500%20Loadout%20press%20release%20July%202004.doc Keppel
Submission+to+the+Human+Rights+and+Equal+Opportunity+Commission.doc Keppel
Rule_Ann-Kiss_Me_Kill_Me.doc Keppel
How%20to%20complete%20the%20PIF%20guidance%20draft%20v0.5.doc Keppel
SUMATRA_INTERNEES_(_Version_1-1.1.0)%20(1)%20(Autosaved)%20copy.doc Keppel
consumersovereigntyantitrustlaw1997.doc Keppel
GMs%20of%20England.doc Keppel
02224.doc Keppel
bijalkvp.doc Keppel
FDI%20in%20India%20%E2%80%93%20Specific%20to%20Registered%20Real%20estate.doc Keppel
News1212.doc Keppel
P020160628597921671163.doc Keppel
Research+Design.doc Keppel
1075751457129.doc Keppel
Long%20Term%20Memory.doc Keppel
CDE-UC%20Approved%20CTE%20Courses%202013-14.doc Keppel
59878_Russia%20100401.doc Keppel
freund.doc Keppel
2009_069.doc Keppel
mentor_app_form_p57-59.doc Keppel
contract_of_employment.doc Keppel
Age

In [51]:
keppel = 'files/Keppel\\'
prudential = 'files/Prudential\\'
## Checked all are included in the if clauses
def fromMapToOpener(fileMap):
    for line in fileMap:
        count = 0
        ID = line[0]
        fileName = line[1]
        compName = line[2]
        ##pdf
        if(fileName.endswith('pdf')):
            if(compName == 'Keppel'):
                count = countPDF(keppel+fileName,'keppel')
            else:
                count = countPDF(prudential+fileName,'prudential')
        ##doc
        elif((fileName.endswith('doc') or fileName.endswith('docx'))):
            if(compName == 'Keppel'):
                count = countDoc(keppel+fileName,'keppel')
            else:
                count = countDoc(prudential+fileName,'prudential')
        ##excel
        elif(fileName.endswith('xls') or fileName.endswith('xlsx')):
            count += 1
        else:
             print('WRONG')
        print(fileName,count)

In [ ]:
fromMapToOpener(filesMap)

WRONG
filename 0
JD_KDCREIT%20-%20Asset%20Mgt_Analyst_2017.pdf 10
5f4c7c69049c32670176932fd1d305271372e912.pdf 43
20100319_214216_BN4_F6859A6425BC8427482576EB00331260.1.pdf 20
businessassociatelist--1-.xlsx 1


WSHA2011%20Winners_8%20Aug.pdf 24
COULD NOT OPEN DOC(X) files/Keppel\sin63964.doc
sin63964.doc 0
KOMTechnology%20StrategyJuly03v3.pdf 21
Keppel%20REIT_AR2016.pdf 0
pre_13sep2010_n2.pdf 4
Dec06.xls 1
04_April_2016.pdf 388
COULD NOT OPEN DOC(X) files/Keppel\bmap_text_8.doc
bmap_text_8.doc 0
KCL_AR2014_Operating_Financial_Review.pdf 281
02_February_2016.pdf 381
nfholdings_1721961_0917.xlsx 1
standcontributorspre1999.xls 1
KLL_SR2011.pdf 0
grants.xls 1
inSYNC_ecopy.pdf 1
COULD NOT OPEN DOC(X) files/Keppel\Dr%20Rob%20Kelly%20-%20Keppel%20Fels%20Loadout%20Speech.doc
Dr%20Rob%20Kelly%20-%20Keppel%20Fels%20Loadout%20Speech.doc 0
Ads.pdf 4
2014 UPDATE: FOCUSING ON LEGAL STATUS, DISABILITY 
Handout-oct14-final-version-17-10-2014.docx 2
REP-1437137611.pdf 655
Civil Law





MUST READ CASES (CIVIL LAW) PERSONS
PALS-Bar-Ops-Pilipinas-Must-Read-Cases-CIVIL-2015.docx 2
om10_mar%20april%202010.pdf 261
2016%20jan-february%20issue.pdf 419
Allianz-Hongkou-SOHO_EN.pdf 12
53_11338.pdf 0
nfa_list.xls 1
COUL

In [28]:
test = "C:/Users/maxpr/Documents/KaggleDathena/files/Keppel/Dr%20Rob%20Kelly%20-%20Keppel%20Fels%20Loadout%20Speech.doc"

In [33]:
f = open(test,'r')
f.read()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 1238: character maps to <undefined>

In [62]:
os.system('antiword -f '+ test+ ' > ' + test[:-3]+'txt')

1

In [60]:
test[:-3]

'C:/Users/maxpr/Documents/KaggleDathena/files/Keppel/Dr%20Rob%20Kelly%20-%20Keppel%20Fels%20Loadout%20Speech.'